In [1]:
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd

In [2]:
# file path to raw data
file_path = 'raw_data/pp-complete.csv'

# define columns names
# Define column names
column_names = ['transaction_id','price', 'transfer_date',
                'postcode', 'property_type', 'new_build_flag',
                'duration', 'primary_address', 'secondary_address',
                'street', 'locality', 'city', 'district',
                'county', 'ppd_category', 'record_status']

# specify data types
dtypes = dict.fromkeys(column_names, 'string')
dtypes.update({'price': 'int64'})

# specify dates to parse
parse_dates = ['transfer_date']

# define columns to keep
usecols = [col for col in column_names if col != 'transaction_id']

ddf = dd.read_csv(file_path, header=None, names=column_names, dtype=dtypes, parse_dates=parse_dates, usecols=usecols)

In [3]:
# compute ddf to pandas dataframe
raw_data_df = ddf.compute()

In [5]:
# drop rows with no postcode
clean_data_df = raw_data_df.dropna(subset=['postcode'])

# drop rows with property type 'O'
clean_data_df = clean_data_df.loc[clean_data_df['property_type']!='O']

# Sort by date
clean_data_df.sort_values(by=['transfer_date'], inplace=True, ignore_index=True)

In [8]:
clean_data_df.head()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,county,ppd_category,record_status
0,125000,1995-01-01,SK10 1QL,D,N,F,28,<NA>,LARK HALL ROAD,MACCLESFIELD,MACCLESFIELD,MACCLESFIELD,CHESHIRE,A,A
1,300000,1995-01-01,SW19 5JU,F,Y,L,1,FLAT 6,SOMERSET ROAD,LONDON,LONDON,MERTON,GREATER LONDON,A,A
2,47000,1995-01-01,SY23 3HE,T,N,F,1,<NA>,MAES YR EFAIL,PENRHYNCOCH,ABERYSTWYTH,CEREDIGION,CEREDIGION,A,A
3,52000,1995-01-01,WA5 1JB,S,N,L,1,<NA>,BUCKINGHAM DRIVE,GREAT SANKEY,WARRINGTON,WARRINGTON,WARRINGTON,A,A
4,49000,1995-01-01,BS11 0JH,D,N,F,21A,<NA>,BARROWMEAD DRIVE,BRISTOL,BRISTOL,BRISTOL,AVON,A,A


In [9]:
clean_data_df['postcode_sector'] = clean_data_df['postcode'].apply(lambda x: x[:x.find(' ')+ 2])

In [10]:
clean_data_df.head()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,county,ppd_category,record_status,postcode_sector
0,125000,1995-01-01,SK10 1QL,D,N,F,28,<NA>,LARK HALL ROAD,MACCLESFIELD,MACCLESFIELD,MACCLESFIELD,CHESHIRE,A,A,SK10 1
1,300000,1995-01-01,SW19 5JU,F,Y,L,1,FLAT 6,SOMERSET ROAD,LONDON,LONDON,MERTON,GREATER LONDON,A,A,SW19 5
2,47000,1995-01-01,SY23 3HE,T,N,F,1,<NA>,MAES YR EFAIL,PENRHYNCOCH,ABERYSTWYTH,CEREDIGION,CEREDIGION,A,A,SY23 3
3,52000,1995-01-01,WA5 1JB,S,N,L,1,<NA>,BUCKINGHAM DRIVE,GREAT SANKEY,WARRINGTON,WARRINGTON,WARRINGTON,A,A,WA5 1
4,49000,1995-01-01,BS11 0JH,D,N,F,21A,<NA>,BARROWMEAD DRIVE,BRISTOL,BRISTOL,BRISTOL,AVON,A,A,BS11 0


### Add region column

In [11]:
# load regions by postcodes
regions_df = pd.read_csv('postcode_data/postcode_areas.csv')

# create dictionary from df
postcode_region_dict = dict(zip(regions_df['postcode_prefix'], regions_df['region']))

# Add postcode prefix column to main df
clean_data_df['postcode_prefix'] = clean_data_df['postcode'].str.extract(r'^([A-Z]+)')

# map postcode prefix to the region using the dictionary
clean_data_df['region'] = clean_data_df['postcode_prefix'].map(postcode_region_dict)

# drop the intermidiate row
clean_data_df.drop(columns=['postcode_prefix'], inplace=True)

# exclude sales in Scotland as limited data available
clean_data_df = clean_data_df.loc[clean_data_df['region'] != 'Scotland']

### Add year column

In [12]:
# extract year from transfer date
clean_data_df['year'] = clean_data_df['transfer_date'].dt.year

#extract month
clean_data_df['month'] = clean_data_df['transfer_date'].dt.month

## Create GeoJSON files

### Convert Shapefiles to GeoJSON

In [13]:
shpfile = gpd.read_file('shapefiles/Sectors.shp')
shpfile.to_file('GeoJSON/allsectors.geojson', driver='GeoJSON')

### Create GeoJSON file for each region

In [14]:
# load postcode prefix CSV
postcode_prefixes_df = pd.read_csv('postcode_data/postcode_areas.csv')

# load the geojson file containing all postcode sector boundaries
geojson_path = 'GeoJSON/allsectors.geojson'

# read the geojson
postcode_sectors_gdf = gpd.read_file(geojson_path)

In [15]:
# Create empty dictionary to store Geodataframes for each region
region_geojson_dict = {}

# extract postcode prefix from 'name' property within geodf
postcode_sectors_gdf['postcode_prefix'] = pd.Series(postcode_sectors_gdf['name']).str.extract(r'^([A-Z]+)')

# merge postcode prefixes df to get regions
merged_df = pd.merge(postcode_sectors_gdf, postcode_prefixes_df, how='left', on= 'postcode_prefix')

In [17]:
# iterate over each unique region
for region in merged_df['region'].unique():
    # filter df for the current region
    region_df = merged_df[merged_df['region'] == region].copy()

    # create a geo df for the current region
    region_gdf = gpd.GeoDataFrame(region_df, geometry='geometry',crs=postcode_sectors_gdf.crs)

    # store the geo df in the dictionary under the region key
    region_geojson_dict[region] = region_gdf

### Export the GeoJSON files

In [18]:
for region, region_gdf in region_geojson_dict.items():
    region_geojson_path = f'GeoJSON/regions/{region}_postcode_sectors.geojson'
    region_gdf.to_file(region_geojson_path, driver='GeoJSON')

## Process data

### Calculate average price for each postcode by year

In [21]:
# group by region, postcode sector and year then calculate avg price
grouped_df = clean_data_df.groupby(['region', 'postcode_sector', 'year']).agg({'price': ['mean', 'count']}).reset_index()

grouped_df.columns = ['region', 'postcode_sector', 'year', 'avg_price', 'volume']

# round the avg price to nearest thousand and convert to int
grouped_df['avg_price'] = grouped_df['avg_price'].round(-3).astype(int)

In [22]:
grouped_df.head()

,region,postcode_sector,year,avg_price,volume
0,East England,PE1 1,1995,46000,11
1,East England,PE1 1,1996,39000,10
2,East England,PE1 1,1997,52000,15
3,East England,PE1 1,1998,49000,21
4,East England,PE1 1,1999,77000,20


In [23]:
# loop though each unique year and save CSV for each
for year in grouped_df['year'].unique():
    # filter df for current year
    year_df = grouped_df[grouped_df['year'] == year]

    # create file path for current year csv
    csv_file_path = f'processed_data/average_price_by_year/region_data_{year}.csv'

    # save the df to csv
    year_df.to_csv(csv_file_path, index=False)

### Calculate average price for each region by year

In [24]:
# group by year, region, property type and calculate average price
region_grouped_df = clean_data_df.groupby(['year', 'region', 'property_type']).agg(avg_price=('price','mean')).round({'avg_price': -3}).astype({'avg_price': int}).reset_index()

# calculate sales volume for each group
region_grouped_df['volume'] = clean_data_df.groupby(['year', 'region', 'property_type']).size().reset_index(name='volume')['volume']

In [25]:
region_grouped_df.head()

,year,region,property_type,avg_price,volume
0,1995,East England,D,72000,6170
1,1995,East England,F,34000,598
2,1995,East England,S,45000,4092
3,1995,East England,T,37000,3131
4,1995,East Midlands,D,78000,21593


In [26]:
# save df to csv file
region_avg_csv_file_path = 'processed_data/region_avg_price/region_avg_prices.csv'

region_grouped_df.to_csv(region_avg_csv_file_path, index=False)

### Calculate delta % year-on-year

In [27]:
# group by region and postcode sector, then calculate delata
delta_df = grouped_df.copy()
delta_df['delta'] = grouped_df.groupby(['region', 'postcode_sector'])['avg_price'].pct_change() * 100

# set the delta to 0 for first year
delta_df.loc[delta_df['year'] == delta_df['year'].min(), 'delta'] = 0

In [28]:
delta_df.head()

,region,postcode_sector,year,avg_price,volume,delta
0,East England,PE1 1,1995,46000,11,0.000000
1,East England,PE1 1,1996,39000,10,-15.217391
2,East England,PE1 1,1997,52000,15,33.333333
3,East England,PE1 1,1998,49000,21,-5.769231
4,East England,PE1 1,1999,77000,20,57.142857


In [29]:
# drop any rows with a null delta
delta_df = delta_df.dropna(subset=['delta'])

# round values to nearest int
delta_df['delta'] = delta_df['delta'].round().astype(int)

In [31]:
# iterate over each year and create a csv for each
for year in delta_df['year'].unique():
    # filter rows for the current year
    year_df = delta_df[delta_df['year'] == year]

    # create csv for the current year
    csv_file_path = f'processed_data/avg_price_delta/avg_price_delta_{year}.csv'
    year_df.to_csv(csv_file_path, index=False)

### Create volume by month for each year

In [35]:
# group by region, postcode_sector, year and month then calculate count of sales
volume_df = clean_data_df.groupby(['region', 'postcode_sector', 'year', 'month']).agg({'price': 'count'}).reset_index()

# rename columns for clarity
volume_df.columns = ['region', 'postcode_sector', 'year', 'month', 'volume']

In [36]:
# loop through each year and create seperate csvs
for year in volume_df['year'].unique():
    # filter df for current year
    year_df = volume_df[volume_df['year'] == year]

    # group by region, month then calculate volume
    total_volume_df = year_df.groupby(['region', 'month']).agg({'volume': 'sum'}).reset_index()

    # create csv from df
    csv_file_path = f'processed_data/volume_by_year/region_total_volume_{year}.csv'
    total_volume_df.to_csv(csv_file_path, index=False)